# Lab 5: Neural Network Classification with scikit-learn

---
## 1.  Notebook Overview

### 1.1 Objective
Build a Multi-Layer Perceptron (MLP) neural network for multi-label tweet topic classification using scikit-learn's `MLPClassifier`.

### 1.2 Prerequisites
This notebook assumes you have already executed:
- **Lab 2**: Data preprocessing → `../Data/tweets_preprocessed_train.parquet`
- **Lab 3** (Sunny): Language modeling
- **Lab 4** (Hikmet): Feature extraction and Naive Bayes baseline

### 1. 3 Architecture
We implement a neural network with:
- **Input layer**: TF-IDF vectorized features
- **Hidden layers**: 128 → 64 → 128 neurons (as specified)
- **Output layer**: 19 neurons (one per topic class)

### 1.4 Data Splits
- **Training**: `../Data/tweets_preprocessed_train.parquet` (from Lab 2)
- **Validation**: `validation_2021` split from HuggingFace
- **Test**: `test_2021` split from HuggingFace

### 1.5 Section Roadmap
1. `Section 2`: Load data from preprocessed parquet and HuggingFace
2. `Section 3`: Feature extraction with TF-IDF
3. `Section 4`: Build and train the MLP neural network
4. `Section 5`: Evaluate on validation and test sets
5.  `Section 6`: Architecture comparison and analysis

---
## 2. Data Loading

Load preprocessed training data from Lab 2 and fetch test/validation splits from HuggingFace.

In [ ]:
%pip install scikit-learn pandas numpy datasets matplotlib seaborn --quiet

In [ ]:
import pandas as pd
import numpy as np
from datasets import load_dataset
import ast
from typing import List
import warnings
warnings.filterwarnings('ignore')

# Constants
TRAIN_DATA_PATH = "../Data/tweets_preprocessed_train.parquet"
RANDOM_STATE = 42

# The 19 topic labels in the dataset
TOPIC_LABELS = [
    'arts_&_culture', 'business_&_entrepreneurs', 'celebrity_&_pop_culture',
    'diaries_&_daily_life', 'family', 'fashion_&_style', 'film_tv_&_video',
    'fitness_&_health', 'food_&_dining', 'gaming', 'learning_&_educational',
    'music', 'news_&_social_concern', 'other_hobbies', 'relationships',
    'science_&_technology', 'sports', 'travel_&_adventure', 'youth_&_student_life'
]
NUM_CLASSES = len(TOPIC_LABELS)

print(f"Number of topic classes: {NUM_CLASSES}")

In [ ]:
def parse_label_vector(value) -> np.ndarray:
    """Parse the label column (binary vector) to numpy array."""
    if isinstance(value, np.ndarray):
        return value. astype(np.float32)
    if isinstance(value, list):
        return np.array(value, dtype=np.float32)
    if isinstance(value, str):
        # Handle string representation like '[0 0 0 1 0 ...]'
        cleaned = value.strip('[]'). split()
        return np.array([float(x) for x in cleaned], dtype=np.float32)
    return np.zeros(NUM_CLASSES, dtype=np. float32)


# Load preprocessed training data from Lab 2
print("Loading preprocessed training data from Lab 2...")
df_train = pd.read_parquet(TRAIN_DATA_PATH)
print(f"Training samples: {len(df_train):,}")
print(f"Columns: {df_train.columns.tolist()}")
df_train.head(3)

In [ ]:
# Load test and validation splits from HuggingFace
print("Loading test and validation data from HuggingFace...")

test_dataset = load_dataset("cardiffnlp/tweet_topic_multi", split="test_2021")
val_dataset = load_dataset("cardiffnlp/tweet_topic_multi", split="validation_2021")

df_test = test_dataset.to_pandas()
df_val = val_dataset.to_pandas()

print(f"Test samples: {len(df_test):,}")
print(f"Validation samples: {len(df_val):,}")

In [ ]:
# Extract labels - use the 'label' column which contains binary vectors
y_train = np.array([parse_label_vector(lbl) for lbl in df_train['label']])
y_test = np.array([parse_label_vector(lbl) for lbl in df_test['label']])
y_val = np.array([parse_label_vector(lbl) for lbl in df_val['label']])

print(f"\nLabel matrix shapes:")
print(f"  y_train: {y_train.shape}")
print(f"  y_test: {y_test.shape}")
print(f"  y_val: {y_val.shape}")

# Verify label distribution
print(f"\nTraining set label distribution:")
for i, label in enumerate(TOPIC_LABELS[:5]):  # Show first 5
    count = int(y_train[:, i].sum())
    print(f"  {label}: {count} ({count/len(y_train)*100:.1f}%)")
print(f"  ...  (showing 5 of {NUM_CLASSES} classes)")

---
## 3. Feature Extraction

Transform text into numerical features using TF-IDF vectorization.

In [ ]:
from sklearn. feature_extraction.text import TfidfVectorizer

MAX_FEATURES = 5000

tfidf_vectorizer = TfidfVectorizer(
    max_features=MAX_FEATURES,
    lowercase=True,
    ngram_range=(1, 2),  # Unigrams and bigrams
    min_df=2,
    max_df=0.95,
)

# Fit on training data only, transform all splits
print("Vectorizing text data...")
X_train = tfidf_vectorizer. fit_transform(df_train['text']. astype(str))
X_test = tfidf_vectorizer.transform(df_test['text'].astype(str))
X_val = tfidf_vectorizer. transform(df_val['text'].astype(str))

print(f"\nFeature matrix shapes:")
print(f"  X_train: {X_train. shape}")
print(f"  X_test: {X_test.shape}")
print(f"  X_val: {X_val.shape}")
print(f"\nVocabulary size: {len(tfidf_vectorizer.get_feature_names_out())}")

---
## 4.  Neural Network Model

### 4.1 Architecture
As specified in the lab requirements:
- **Input layer**: Determined by TF-IDF features
- **Hidden layer 1**: 128 neurons
- **Hidden layer 2**: 64 neurons
- **Hidden layer 3**: 128 neurons
- **Output layer**: 19 neurons (one per topic)

### 4.2 Why no early_stopping?
We have dedicated validation (`validation_2021`) and test (`test_2021`) splits from HuggingFace. There's no need for scikit-learn to internally split the training data for early stopping validation.

In [ ]:
from sklearn.neural_network import MLPClassifier
from sklearn.multioutput import MultiOutputClassifier
import time

# Architecture as specified: 128 -> 64 -> 128
HIDDEN_LAYERS = (128, 64, 128)

print("=" * 60)
print("NEURAL NETWORK CONFIGURATION")
print("=" * 60)
print(f"Input layer: {X_train.shape[1]} features")
print(f"Hidden layers: {HIDDEN_LAYERS}")
print(f"Output layer: {NUM_CLASSES} classes")
print("=" * 60)

# Create MLP classifier
# No early_stopping - we use HuggingFace's validation_2021 for model selection
mlp_base = MLPClassifier(
    hidden_layer_sizes=HIDDEN_LAYERS,
    activation='relu',
    solver='adam',
    alpha=0.0001,
    batch_size='auto',
    learning_rate='adaptive',
    learning_rate_init=0.001,
    max_iter=200,
    random_state=RANDOM_STATE,
    verbose=True
)

# Wrap for multi-label classification
mlp_classifier = MultiOutputClassifier(mlp_base, n_jobs=-1)

print("\n✓ Model created successfully")

In [ ]:
# Train the model
print("\n" + "=" * 60)
print("TRAINING")
print("=" * 60)

start_time = time.time()
mlp_classifier.fit(X_train, y_train)
training_time = time. time() - start_time

print(f"\n✓ Training completed in {training_time:.2f} seconds")

# Check convergence
converged = sum(1 for est in mlp_classifier.estimators_ if est.n_iter_ < est.max_iter)
print(f"✓ {converged}/{NUM_CLASSES} classifiers converged before max_iter")

---
## 5. Model Evaluation

Evaluate on both validation and test sets.

In [ ]:
from sklearn.metrics import (
    accuracy_score, f1_score, precision_score, recall_score,
    hamming_loss, classification_report
)

# Predictions
y_pred_val = mlp_classifier.predict(X_val)
y_pred_test = mlp_classifier.predict(X_test)

def evaluate(y_true, y_pred, split_name):
    """Print evaluation metrics for a given split."""
    print(f"\n{'='*60}")
    print(f"{split_name. upper()} SET METRICS")
    print("="*60)
    print(f"{'Subset Accuracy':<20}: {accuracy_score(y_true, y_pred):.4f}")
    print(f"{'Hamming Loss':<20}: {hamming_loss(y_true, y_pred):.4f}")
    print(f"{'Micro F1':<20}: {f1_score(y_true, y_pred, average='micro', zero_division=0):.4f}")
    print(f"{'Macro F1':<20}: {f1_score(y_true, y_pred, average='macro', zero_division=0):.4f}")
    print(f"{'Weighted F1':<20}: {f1_score(y_true, y_pred, average='weighted', zero_division=0):.4f}")
    print(f"{'Micro Precision':<20}: {precision_score(y_true, y_pred, average='micro', zero_division=0):.4f}")
    print(f"{'Micro Recall':<20}: {recall_score(y_true, y_pred, average='micro', zero_division=0):. 4f}")

# Evaluate on validation set (for model selection)
evaluate(y_val, y_pred_val, "Validation")

# Evaluate on test set (final performance)
evaluate(y_test, y_pred_test, "Test")

In [ ]:
# Per-class performance on test set
print("\n" + "=" * 60)
print("PER-CLASS PERFORMANCE (Test Set)")
print("=" * 60)

print(classification_report(
    y_test, y_pred_test,
    target_names=TOPIC_LABELS,
    zero_division=0
))

In [ ]:
import matplotlib.pyplot as plt

# Visualize per-class F1 scores
per_class_f1 = f1_score(y_test, y_pred_test, average=None, zero_division=0)

fig, ax = plt.subplots(figsize=(12, 6))
bars = ax.barh(TOPIC_LABELS, per_class_f1, color='steelblue')
ax.set_xlabel('F1 Score')
ax.set_title('Per-Class F1 Scores (Test Set)')
ax.set_xlim(0, 1)

for bar, score in zip(bars, per_class_f1):
    ax.text(bar.get_width() + 0.01, bar.get_y() + bar. get_height()/2,
            f'{score:.3f}', va='center', fontsize=9)

plt.tight_layout()
plt. show()

---
## 6. Architecture Comparison

Compare different hidden layer configurations.

In [ ]:
architectures = [
    (64,),               # Single layer
    (128, 64),           # Two layers
    (128, 64, 128),      # Three layers (baseline)
    (256, 128, 64),      # Wider first layer
]

print("=" * 60)
print("ARCHITECTURE COMPARISON")
print("=" * 60)

results = []

for arch in architectures:
    print(f"\nTraining: {arch}")
    
    mlp = MLPClassifier(
        hidden_layer_sizes=arch,
        activation='relu',
        solver='adam',
        alpha=0.0001,
        learning_rate='adaptive',
        max_iter=100,
        random_state=RANDOM_STATE,
        verbose=False
    )
    
    clf = MultiOutputClassifier(mlp, n_jobs=-1)
    
    start = time.time()
    clf.fit(X_train, y_train)
    train_time = time. time() - start
    
    # Evaluate on VALIDATION set (for model selection)
    y_pred = clf.predict(X_val)
    
    f1_micro = f1_score(y_val, y_pred, average='micro', zero_division=0)
    f1_macro = f1_score(y_val, y_pred, average='macro', zero_division=0)
    
    results.append({
        'Architecture': str(arch),
        'Micro F1': f1_micro,
        'Macro F1': f1_macro,
        'Train Time (s)': train_time
    })
    
    print(f"  Val Micro F1: {f1_micro:.4f}, Macro F1: {f1_macro:.4f}, Time: {train_time:.1f}s")

results_df = pd. DataFrame(results)
print("\n" + "=" * 60)
print("SUMMARY (Validation Set)")
print("=" * 60)
print(results_df.to_string(index=False))

In [ ]:
# Visualization
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

x = range(len(results_df))
width = 0.35

axes[0]. bar([i - width/2 for i in x], results_df['Micro F1'], width, label='Micro F1', color='steelblue')
axes[0].bar([i + width/2 for i in x], results_df['Macro F1'], width, label='Macro F1', color='coral')
axes[0].set_xticks(x)
axes[0].set_xticklabels(results_df['Architecture'], rotation=45, ha='right')
axes[0].set_ylabel('F1 Score')
axes[0].set_title('F1 Scores by Architecture (Validation Set)')
axes[0].legend()
axes[0].set_ylim(0, 1)

axes[1].bar(x, results_df['Train Time (s)'], color='seagreen')
axes[1].set_xticks(x)
axes[1]. set_xticklabels(results_df['Architecture'], rotation=45, ha='right')
axes[1].set_ylabel('Training Time (seconds)')
axes[1].set_title('Training Time by Architecture')

plt.tight_layout()
plt.show()

---
## 7. Sample Predictions

In [ ]:
def binary_to_labels(binary_vec):
    """Convert binary vector to label names."""
    return [TOPIC_LABELS[i] for i, val in enumerate(binary_vec) if val == 1] or ['[none]']

print("=" * 60)
print("SAMPLE PREDICTIONS (Test Set)")
print("=" * 60)

np.random.seed(RANDOM_STATE)
sample_idx = np.random. choice(len(df_test), size=10, replace=False)

for idx in sample_idx:
    text = df_test['text'].iloc[idx]
    true = binary_to_labels(y_test[idx])
    pred = binary_to_labels(y_pred_test[idx])
    
    match = "✓" if set(true) == set(pred) else "✗"
    
    print(f"\n{match} Sample {idx}:")
    print(f"   Text: {str(text)[:80]}...")
    print(f"   True: {true}")
    print(f"   Pred: {pred}")

---
## 8.  Conclusion

### 8.1 Summary
We built an MLP neural network with the specified 128-64-128 architecture for multi-label tweet classification.

### 8.2 Key Design Decisions
- **No early_stopping**: We use HuggingFace's `validation_2021` split for model selection and `test_2021` for final evaluation
- **MultiOutputClassifier wrapper**: Handles multi-label by training 19 independent binary classifiers
- **TF-IDF features**: Captures term importance relative to the corpus

### 8.3 Limitations of scikit-learn's MLP
- No GPU acceleration
- No dropout or batch normalization
- Limited to fully-connected layers

### 8.4 Next Steps
- Tune regularization (alpha parameter)
- Try different activation functions
- Consider deep learning frameworks (PyTorch/TensorFlow) for more flexibility

In [ ]:
# Final summary
test_f1_micro = f1_score(y_test, y_pred_test, average='micro', zero_division=0)
test_f1_macro = f1_score(y_test, y_pred_test, average='macro', zero_division=0)

print("\n" + "=" * 60)
print("FINAL MODEL SUMMARY")
print("=" * 60)
print(f"Architecture: Input({X_train. shape[1]}) → 128 → 64 → 128 → Output({NUM_CLASSES})")
print(f"Training samples: {len(y_train):,}")
print(f"Validation samples: {len(y_val):,}")
print(f"Test samples: {len(y_test):,}")
print(f"Test Micro F1: {test_f1_micro:.4f}")
print(f"Test Macro F1: {test_f1_macro:. 4f}")
print("=" * 60)